# Preprocesado e ingeniería de características

## Procesado de los datos

El conjunto de datos sobre reconocimiento de vinos está incluido en *Scikit-learn*, se obtiene usando la función `load_wine` incluida en la librería `sklearn.datasets`. Este conjunto de datos contiene 178 ejemplos de distintas variedades de vino, con 13 características y tres clasificaciones posibles.

In [162]:
from sklearn.datasets import load_wine

wine = load_wine()

Este conjunto de datos es un diccionario con varios campos:
* `data`: Es el conjunto de datos, se trata de un array en el que cada componente es un array con las características de cada instancia.
* `target`: Es el conjunto de valores de clasificación para cada instancia. Es un array del mismo tamaño que `data`, en el que se indica el valor de clasificación de cada instancia, en el mismo orden en que éstas se encuentran en el array `data`.
* `DESCR`: Es una descripción del conjunto de datos.
* `target_names`: Es un array con los nombres de cada valor de clasificación.
* `feature_names`: Es un array con los nombres de cada característica.

Almacenamos los datos en las variables `X_data`, `y_data`, `X_names` e `y_names`.

In [163]:
X_data, y_data, X_names, y_names = \
    wine.data, wine.target, wine.feature_names, wine.target_names

## Contenido del ejercicio

El ejercicio consiste en aplicar distintas estrategias de reducción de dimensionalidad y comparar los resultados
* Implementar una función para realizar la eliminación recursiva de características tal y como se ha explicado en teoría, para reducir el número de características hasta un valor dado. (No es igual al método implementado por la clase RFE incluido en *Scikit-learn*)
* Realizar selección de características basada en modelos para reducir el número de características del conjunto de datos. Investigar los parámetros de la clase SelectFromModel para reducir el número de características hasta un valor dado.
* Estudiar las características del conjunto de datos mediante análisis de componentes principales. Reducir el número de características hasta un valor dado.

El **desarrollo tiene que estar razonado**, indicando en cada apartado qué se está haciendo, **demostrando así el conocimiento adquirido en este módulo**. ¿Qué conclusiones puedes sacar sobre las características de este conjunto de datos?

### Ejercicio 1
#### Implementar una función para realizar la eliminación recursiva de características tal y como se ha explicado en teoría, para reducir el número de características hasta un valor dado. (No es igual al método implementado por la clase RFE incluido en *Scikit-learn*)

Definimos la función *removeFeature* que recibe los siguientes parámetros:
- *X_data*: Conjunto de datos
- *y_data*: Valores de clasificación
- *num_features*: Número de características a eliminar
- *X_names_feature*: Nombres de las características
- *removedFeaturesList*: Lista con las características que se van eliminando. Inicialmente es una lista vacía para se utilizará al llamar a la función de forma recursiva para almacenar el nombre de las características que se van eliminando.

Esta función va eliminando características y comprobando el rendimiento que se obtiene al eliminarla. Finalmente eliminará con la que mejores resultados se consigan. Así hasta eliminar el número de características que se indique en el parámetro *num_features*.

In [164]:
from sklearn.svm import LinearSVC
import numpy as np
def removeFeature(X_data, y_data, num_features, X_names_feature, removedFeaturesList):
    model = LinearSVC(random_state=486, max_iter=10000).fit(X_data, y_data)
    bestScore = model.score(X_data,y_data)
    bestX_data = np.copy(X_data)
    X_features = np.copy(X_names_feature)

    removedFeature = ''
    for i in range(X_data.shape[1]):
        X_data_aux = np.copy(X_data)
        X_data_reduced = np.delete(X_data_aux, i, 1)
        model = LinearSVC(max_iter=10000).fit(X_data_reduced, y_data)
        score = model.score(X_data_reduced, y_data)
        if score > bestScore:
            bestScore = score
            bestX_data = X_data_reduced
            removedFeature = X_names_feature[i]
            X_names_feature_aux = np.copy(X_names_feature)
            X_features = np.delete(X_names_feature_aux, i, 0)

    if removedFeature != '':
        removedFeaturesList.append(removedFeature)
        if len(removedFeaturesList) < num_features:
            return removeFeature(bestX_data, y_data, num_features, X_features, removedFeaturesList)
        else:
            return bestX_data, X_features, removedFeaturesList, bestScore
    else:
        return bestX_data, X_features, removedFeaturesList, bestScore

Vamos a utilizar la función que se ha definido para eliminar hasta 2 características y a ver qué resultados obtenemos.

In [165]:
X_data_result, X_names_result, removedFeatures_result, bestScore = removeFeature(X_data, y_data, 2, X_names, [])
print('Se han eliminado las siguientes {} características: {}'.format(len(removedFeatures_result), removedFeatures_result))
print('Rendimiento obtenido al eliminar las características: {:.3f}'.format(bestScore))
print('Forma del dataset resultante: ', X_data_result.shape)
print('Características finales: ', X_names_result)

/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

Se han eliminado las siguientes 2 características: ['nonflavanoid_phenols', 'alcohol']
Rendimiento obtenido al eliminar las características: 0.966
Forma del dataset resultante:  (178, 11)
Características finales:  ['malic_acid' 'ash' 'alcalinity_of_ash' 'magnesium' 'total_phenols'
 'flavanoids' 'proanthocyanins' 'color_intensity' 'hue'
 'od280/od315_of_diluted_wines' 'proline']


/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Ejercicio 2
#### Realizar selección de características basada en modelos para reducir el número de características del conjunto de datos. Investigar los parámetros de la clase SelectFromModel para reducir el número de características hasta un valor dado.

Entre los distintos parámetros que se pueden utilizar en la clase `SelectFromModel`, vamos a utilizar *max_features* para indicar el máximo número de características a seleccionar.<br>Primero vamos a probar indicando como máximo 11 características y luego como máximo 4. Vamos a seleccionarlas en ambos casos a partir de un modelo lineal con regularización L1.

In [166]:
from sklearn.feature_selection import SelectFromModel
linear_svm = LinearSVC(C=0.01, penalty="l1", dual=False)
score = linear_svm.fit(X_data,y_data).score(X_data,y_data)

select = SelectFromModel(linear_svm, max_features = 11)
select.fit(X_data,y_data)
X_data_11 = select.transform(X_data)
score_11 = linear_svm.fit(X_data_11,y_data).score(X_data_11,y_data)

select_4 = SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False), max_features = 4)
select_4.fit(X_data,y_data)
X_data_4 = select_4.transform(X_data)
score_4 = linear_svm.fit(X_data_4,y_data).score(X_data_4,y_data)

print('Forma del conjunto inicial es: {}'.format(X_data.shape))
print('Rendimiento del conjunto inicial: {:.3f}'.format(score))
print('Forma del conjunto indicando como máximo 11 características: {}'.format(X_data_11.shape))
print('Rendimiento del conjunto indicando como máximo 11 características: {:.3f}'.format(score_11))
print('Forma del conjunto indicando como máximo 4 características: {}'.format(X_data_4.shape))
print('Rendimiento del conjunto indicando como máximo 4 características: {:.3f}'.format(score_4))

Forma del conjunto inicial es: (178, 13)
Rendimiento del conjunto inicial: 0.904
Forma del conjunto indicando como máximo 11 características: (178, 5)
Rendimiento del conjunto indicando como máximo 11 características: 0.904
Forma del conjunto indicando como máximo 4 características: (178, 4)
Rendimiento del conjunto indicando como máximo 4 características: 0.865


Podemos ver que aunque indiquemos que como máximo son 11 características, ha seleccionado 5 y obtenemos el mismo rendimiento que con el conjunto inicial que tiene 13 características. En cambio, al obligarle que como máximo tenga 4, obtenemos un conjunto con ese máximo y con una disminución en el rendimiento.

### Ejercicio 3
#### Estudiar las características del conjunto de datos mediante análisis de componentes principales. Reducir el número de características hasta un valor dado.
En este ejercicio vamos a aplicar el análisis de componentes principales. En primer lugar los vamos a aplicar al conjunto de datos con todas las características y así ver el grado de importancia de las componentes (autovalores asociados a las componentes principales)

In [167]:
from sklearn.decomposition import PCA
pca = PCA(n_components=13)
pca.fit(X_data)
X_pca = pca.transform(X_data)
pca.explained_variance_

array([9.92017895e+04, 1.72535266e+02, 9.43811370e+00, 4.99117861e+00,
       1.22884523e+00, 8.41063869e-01, 2.78973523e-01, 1.51381266e-01,
       1.12096765e-01, 7.17026032e-02, 3.75759789e-02, 2.10723661e-02,
       8.20370314e-03])

Observamos que las componentes con mas importancia son las 2 primeras. Siguiendo el ejemplo anterior, vamos a aplicar este análisis primero eliminando 2 características, luego eliminando 9 y finalmente dejando una única característica. Para cada caso vamos a el rendimiento que obtenemos.

In [168]:
linear_svm = LinearSVC(C=0.01, penalty="l1", dual=False)
pca_11 = PCA(n_components=11)
pca_11.fit(X_data)
X_pca_11 = pca_11.transform(X_data)
score_11 = linear_svm.fit(X_pca_11,y_data).score(X_pca_11,y_data)

pca_4 = PCA(n_components=4)
pca_4.fit(X_data)
X_pca_4 = pca_4.transform(X_data)
score_4 = linear_svm.fit(X_pca_4,y_data).score(X_pca_4,y_data)

pca_1 = PCA(n_components=1)
pca_1.fit(X_data)
X_pca_1 = pca_1.transform(X_data)
score_1 = linear_svm.fit(X_pca_1,y_data).score(X_pca_1,y_data)

score = linear_svm.fit(X_data,y_data).score(X_data,y_data)
print('Rendimiento del conjunto inicial: {:.3f}'.format(score))
print('Rendimiento del conjunto eliminando 2 características: {:.3f}'.format(score_11))
print('Rendimiento del conjunto eliminando 9 características: {:.3f}'.format(score_4))
print('Rendimiento del conjunto eliminando 12 características: {:.3f}'.format(score_1))

Rendimiento del conjunto inicial: 0.904
Rendimiento del conjunto eliminando 2 características: 0.910
Rendimiento del conjunto eliminando 9 características: 0.910
Rendimiento del conjunto eliminando 12 características: 0.685


Vemos que eliminando características mejora levemente el rendimiento, aunque no hay diferencia entre eliminar 9 o 2 características. En cambio, si solo dejamos una única carecterística, el rendimiento empeora notablemente.<br><br>

Como conclusión, viendo los resultados obtenidos tras aplicar distintas técnicas para reducir la dimensionalidad, hemos comprobado que este conjunto de datos tiene bastantes características que no aportan mucho y que podemos eliminar, para de esta forma obtener mejores resultados pero con un problema más sencillo, pues se ha reducido la dimensionalidad.